In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
load("environment/real_datasets/GSE100505/mapDf.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/treeCons.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/treeCons0.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/treeCor.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/treeMeanInf0.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/yAggCons.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/yAggCons0.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/yAggCor.RData")
load("environment/real_datasets/GSE100505/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/comp_trees/yAggMeanInf0.RData")
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))


In [ ]:
wlfcDiff <- 0
txpDiff <- NULL

In [ ]:
dfDepth <- data.frame(matrix(nrow=0, ncol=4))
colnames(dfDepth) <- c("Type", "Value", "nCuts", "gamma")

l <- length(treeCons$tip)
trees <- list(treeCons, treeCons0, treeMeanInf0, treeCor)
names(trees) <- c("Consensus", "Consensus0", "MeanInf0", "Cor")
yAgg <- list(yAggCons, yAggCons0, yAggMeanInf0, yAggCor)
names(yAgg) <- c("Consensus", "Consensus0", "MeanInf0", "Cor")

dCutsD <- vector(mode = "list", length=4)
names(dCutsD) <- paste("gamma=", c(0.1,0.5,1,5), sep="")
for(j in seq_along(dCutsD)) {
    gamma <- c(0.1,0.5,1,5)[j]
    dCutsD[[j]] <- vector(mode="list", length(trees))
    names(dCutsD[[j]]) <- names(trees)
    for(i in seq(trees)) {
        globArr <- rep(-100, l+trees[[i]]$Nnode)
        descL <- sapply(Descendants(trees[[i]], seq(l+trees[[i]]$Nnode)), length)
        infReps <- assays(yAgg[[i]])[grep("infRep", assayNames(yAgg[[i]]))]
        infReps <- abind::abind(as.list(infReps), along = 3)
        lfc <- getLog2FC(infReps, colData(yAgg[[i]])[["condition"]])
        obj <- abs(lfc/mcols(yAgg[[i]])[["meanInfRV"]]) - gamma*node.depth(trees[[i]],2)
        val <- findMaxSum(trees[[i]], obj, l+1, descL)
        gc()
        dCutsD[[j]][[i]] <- findCuts(trees[[i]], globArr, obj, l+1, descL)
        gc()
        dfDepth <- rbind(dfDepth, data.frame(Type = names(trees)[i], Value = val,
               nCuts = length(dCutsD[[j]][[i]]), gamma=gamma))
    }

}

